# Overview of libraries

We are trying out various freely available linear algebra libraries

## Lapack

[Lapack](https://netlib.org/lapack/) is the legacy linear algebra library. It contains matrix operations, factorizations, linear systems, eigen-system calculation, and more. It is a quasi standard, highly optimized implementations are available for all major computer systems. 

Some implementations are:
  - Intel MKL (for Intel processors + compatible)
  - OpenBlas (open source, for many architectures)
  - Accelerator framework (for Apple systems)


If not available, install Lapack on your computer.

Add this line to your *CMakeLists.txt* file:
```
find_package(LAPACK REQUIRED)
```
Run cmake, and look for output showing *BLAS* and *LAPACK*.

Lapack was written in Fortran, and automatically translated to C using the f2c compiler. There is one C-header file [clapack.h](https://netlib.org/clapack/clapack.h) which one includes like that:


```cpp
#include <complex>

typedef int integer;
typedef integer logical;
typedef float real;
typedef double doublereal;
typedef std::complex<float> singlecomplex;
typedef std::complex<double> doublecomplex;

// Windows SDK defines VOID in the file WinNT.h
#ifndef VOID
    typedef void VOID;
#endif
typedef int ftnlen;
typedef int L_fp;  // ?


extern "C" {
#include <clapack.h>
}
```

With origin in Fortran-times, all functions have a short name from abbreviations, like
**D**ouble-precission **GE**neral **M**atrix **M**atrix multiplication:
```cpp
int dgemm_ (char *transa, char *transb, integer *m, integer *
           n, integer *k, doublereal *alpha, doublereal *a, integer *lda, 
           doublereal *b, integer *ldb, doublereal *beta, doublereal *c__, 
           integer *ldc);
```
[Documentation](https://www.netlib.org/lapack/explore-html/d1/d54/group__double__blas__level3_gaeda3cbd99c8fb834a60a6412878226e1.html)

Since function calls with many arguments are not very clear, we prefer to wrap the low-level Lapack call into a function based on our `MatrixView` classes. In Fortran/Lapack matrices are stored column-major. If a matrix is stored row-wise, we interpret it as a transpose operation:

```cpp
// c = a*b
template <ORDERING OA, ORDERING OB>
void MultMatMatLapack (MatrixView<double, OA> a,
                       MatrixView<double, OB> b,
                       MatrixView<double, ColMajor> c)
{
  char transa = (OA == ColMajor) ? 'N' : 'T';
  char transb = (OB == ColMajor) ? 'N' : 'T'; 
  
  integer n = c.Height();
  integer m = c.Width();
  integer k = a.Width();
  if (n == 0 || m == 0) return 0;
  
  double alpha = 1.0;
  double beta = 0;
  integer lda = a.Dist();
  integer ldb = b.Dist();
  integer ldc = c.Dist();

  int errcode =
    dgemm_ (&transa, &transb, &n, &m, &k, &alpha, 
            a.Data(), &lda, b.Data(), &ldb, &beta, c.Data(), &ldc);
  if (errcode != 0)
     throw exception ("Lapack-dgemm error "+to_string(errcode));
}
                       
template <ORDERING OA, ORDERING OB>
void MultMatMatLapack (MatrixView<double, OA> a,
                       MatrixView<double, OB> b,
                       MatrixView<double, RowMajor> c)
{
  MultMatMatLapack (Trans(b), Trans(a), Trans(c));
}
```           

## C++ libraries

There are a couple of modern C++ linear algebra libraries:

* [eigen](https://eigen.tuxfamily.org)

* [blaze](https://bitbucket.org/blaze-lib/blaze)

* [Fastor](https://github.com/romeric/Fastor)

In the core, most of them fall back to legacy Lapack for large matrices.

## Exercise:

* Look up Lapack functions for
  - LU decomposition (`dgetrf`, `dgetri`) 
  - QR decomposition using Householder reflections (`dgeqrf`, `dorgqr`)
  - Eigenvalue calculation (`dgeev`)
    
  and write C++ wrapper functions for calling the Lapack functions.
  Then, wrap your C++ functions to Python and test them.

* measure performance of Lapack matrix-matrix multiplication, and compare to your expression template matrix-matrix multiplication.